# Combining Precipitation and Elevation from Raster

In this notebook, I will create datasets needed for my analysis. This will involve combining all monthly datasets together, combining all daily datasets together and creating an elevation data file.

## Import libraries

We need `ftplib` for managing FTP connections and `rasterio` for working with rasters.

In [1]:
import os
import ssl
import json

import numpy as np
import pandas as pd

import rasterio
import geopandas as gpd
from ftplib import FTP_TLS

## Load centers

We load in the centers file so we can use the coordinates for extracting data.

In [2]:
centers_data = pd.read_csv("../data/centers_data/combined_centers_data.csv")
centers_data.head()

,ID,NAME,ADDRESS,STATE,ZIP,LATITUDE,LONGITUDE,TYPE
0,898356,ARBOR HEALTH MORTON GENERAL HOSPITAL,521 ADAMS ST,WA,98356,46.555675,-122.280354,HOSPITAL
1,2912601,VASSAR BROTHERS MEDICAL CENTER,45 READE PL,NY,12601,41.693859,-73.935786,HOSPITAL
2,3214215,ERIE COUNTY MEDICAL CENTER,462 GRIDER ST,NY,14215,42.925077,-78.831146,HOSPITAL
3,5514607,STRONG MEMORIAL HOSPITAL,601 ELMWOOD AVE,NY,14607,43.122809,-77.624259,HOSPITAL
4,6411030,NORTH SHORE UNIVERSITY HOSPITAL,300 COMMUNITY DR,NY,11030,40.777636,-73.701611,HOSPITAL


I'll extract the coordinates which will be needed for getting the corresponding data values.

In [3]:
coords = [(x,y) for x, y in zip(centers_data["LONGITUDE"], centers_data["LATITUDE"])]

## Load credentials

Access to data is through an authorized ftp connections, so we need to load in our credentials. I have kept the credentials inside the file **eosdis_credentials.json** as a json format which I read in.

In [4]:
# Open credentials file
credentials_file = open("../eosdis_credentials.json")

# Load credentials
credentials = json.load(credentials_file)
username = credentials["username"]
password = credentials["password"]
  
# Close the file
credentials_file.close()

## Set up FTP

We will now set up the FTP connection which we can use to get the data from the server. The set up instructions are taken from [GPM](https://gpm.nasa.gov/data/directory/imerg-final-run-pps-research-gis) download instructions for FTP.

In [21]:
ftp_site = "arthurhouftps.pps.eosdis.nasa.gov"
FTP_TLS.ssl_version = ssl.PROTOCOL_TLSv1_2
ftps = FTP_TLS()
ftps.connect(ftp_site, 21)
ftps.login(username, password)
ftps.prot_p()

'200 Protection set to Private'

## Get data

We will now use the FTP connection to get the TIF files, extract data for the health centers and create our dataframe.

### Monthly data download

The following function enables us to download monthly precipitation data for a given date range for a given set of coordinates.

In [9]:
def get_monthly_data(coords = [], ids = [], start_year = 2010, end_year = 2020):
    
    # If year is less than 2001, it won't work
    if start_year < 2001:
        print("ERROR: Start year must be more than 2000")
        return
     
    # If year is more than 2020, it won't work
    if end_year > 2020:
        print("ERROR: End year must be less than 2021")
        return
    
    # If end year is less than start year, it won't work
    if end_year < start_year:
        print("ERROR: End year must be after start year")
        return
        
    # Start creating a resultant dataframe
    resultant_data = pd.DataFrame({"ID": ids, "temp": range(len(coords))})
    
    # For each year
    for year in range(start_year, end_year + 1):
        
        # For each month in the current year
        for month in ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]:
        
            # Get the TIF file and save to local temporary file
            f = open("temporary_tif.tif", "wb")
            ftps.retrbinary('RETR pub/gpmdata/' + str(year) + '/' + month + '/01/gis/3B-MO-GIS.MS.MRG.3IMERG.' + str(year) + month + '01-S000000-E235959.' + month + '.V06B.tif', f.write)
            f.close()
            
            # Open that file
            src = rasterio.open('temporary_tif.tif')
            
            # Get data from rastor
            resultant_data["temp"] = [x for x in src.sample(coords)]
            resultant_data[month + "_" + str(year)] = resultant_data["temp"].apply(lambda x: x[0])
            
            # Scale down by a factor of 1000 (documentation states the values are scaled up)
            resultant_data[month + "_" + str(year)] = resultant_data[month + "_" + str(year)].div(1000)
            
            # Remove the temporary file
            os.remove("temporary_tif.tif")
            
        # Print when data is loaded for a given year
        print("Completed extraction for {}".format(year))
        
    # Return final dataframe
    return resultant_data.drop(["temp"], axis = 1)

I will download the monthly precipitation data for the years 2005 to 2020 and save it.

In [10]:
prec_for_centers = get_monthly_data(coords, centers_data["ID"], 2005, 2020)
prec_for_centers.to_csv("../data/precipitation_data/prec_for_centers_monthly.csv", index = False)

Completed extraction for 2005
Completed extraction for 2006
Completed extraction for 2007
Completed extraction for 2008
Completed extraction for 2009
Completed extraction for 2010
Completed extraction for 2011
Completed extraction for 2012
Completed extraction for 2013
Completed extraction for 2014
Completed extraction for 2015
Completed extraction for 2016
Completed extraction for 2017
Completed extraction for 2018
Completed extraction for 2019
Completed extraction for 2020


### Adding elevation data for monthly data

Finally, I'll combine the elevation data I got from The **World TIF**, which is taken from [ASTER Global Digital Elevation Map](https://asterweb.jpl.nasa.gov/gdem.asp). The values are in metres.

In [24]:
def get_elevation_data(coords, ids, elevation_file):
      
    # Create resultant dataframe
    resultant_data = pd.DataFrame({"ID": ids})
    
    # Read the TIF file
    src = rasterio.open(elevation_file)
    
    # Get elevation data
    resultant_data["elevation"] = [x for x in src.sample(coords)]
    resultant_data["elevation"] = resultant_data["elevation"].apply(lambda x: x[0])
        
    # Return resultant data
    return resultant_data

In [25]:
elevation_for_centers = get_elevation_data(coords, centers_data["ID"], "../data/elevation_data/GDEM-10km-BW.tif")
elevation_for_centers.to_csv("../data/elevation_data/elevation_for_centers.csv", index = False)